In [40]:
import pandas as pd
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

s = requests.Session()

retries = Retry(total=5,
                status_forcelist=[429, 500, 502, 503, 504])

s.mount('http://', HTTPAdapter(max_retries=retries))

In [41]:
resp = s.get(f'https://api.squiggle.com.au/?q=games;year={2024};round={24}',headers={"User-Agent": "brody1geore@gmail.com"})
data = resp.json()
df = pd.json_normalize(data["games"])
a = {'Crows': 'Adelaide','Lions': 'Brisbane Lions', 'Blues': 'Carlton', 'Magpies': 'Collingwood', 'Bombers': 'Essendon', 'Dockers': 'Fremantle', 'Cats': 'Geelong', 'Suns': 'Gold Coast', 'Giants': 'Greater Western Sydney', 'Hawks': 'Hawthorn', 'Demons': 'Melbourne', 'Kangaroos': 'North Melbourne', 'Power': 'Port Adelaide', 'Tigers': 'Richmond', 'Saints': 'St Kilda', 'Swans': 'Sydney', 'Eagles': 'West Coast', 'Bulldogs': 'Western Bulldogs'}
df2= pd.read_csv('2024-r24-team_ladder.csv')

In [55]:
df2['Team']= df2['Team'].map(a).fillna(df2['Team'])
# df2.drop(columns=['Streak'], inplace=True)
csv_final = df2[["Position", "Team"]]
csv_final = csv_final.rename(columns={'Team':'hteam', 'Position':'Rank'})
api_final= df[["hteam", "hscore", "ateam", "ascore", "unixtime"]]
df3 = api_final.merge(right=csv_final, how='inner', on='hteam').rename(columns={'Rank':'hteam rank'})
df3 = df3.merge(right=csv_final, how='inner', left_on='ateam', right_on='hteam').rename(columns={'Rank':'ateam rank'})
df3 = df3.drop(df3.filter(regex='_y$').columns, axis=1).rename(columns={'hteam_x':'hteam'})
df3.head()
# df3.csv_final.merge(right=api_final, how='inner', on='hteam')

,hteam,hscore,ateam,ascore,unixtime,hteam rank,ateam rank
0,Melbourne,57,Collingwood,103,1724406000,14,9
1,Geelong,168,West Coast,75,1724471100,3,16
2,Richmond,66,Gold Coast,94,1724472600,18,13
3,Hawthorn,170,North Melbourne,46,1724481300,7,17
4,Brisbane Lions,87,Essendon,67,1724491500,5,11
